In [ ]:
library('ggplot2')
library('reshape2')

In [ ]:
sig2s = c(2,5,10)
mus = c(1,5,20)
N = 100

In [ ]:
Ns = rep(N,3)
Ns

In [ ]:
g1 <- data.frame(group=1,x = rnorm(Ns[1],mus[1],sig2s[1]))
g2 <- data.frame(group=2,x = rnorm(Ns[2],mus[2],sig2s[2]))
g3 <- data.frame(group=3,x = rnorm(Ns[3],mus[3],sig2s[3]))
d = rbind(g1,g2,g3)
d$group <- as.factor(d$group)
d = d[sample(nrow(d)),]

In [ ]:
head(d)

In [ ]:
ggplot(data=d,mapping=aes(x=x,color=group,group=group))+
    geom_density()+
    geom_point(mapping=aes(x=x,y=0))

In [ ]:
class=1
  
subd = d[d$group==class,]
head(subd)

In [ ]:
# mu1_hat
muhat = mean(subd$x)
muhat

In [ ]:
# pi_hat
pi_hat = mean(d$group==class)
pi_hat

In [ ]:
var_hat = var(subd$x)
var_hat

In [ ]:
x0 = 1

In [ ]:
dnorm(x0,mean=muhat,sd=sqrt(var_hat))*pi_hat

In [ ]:
delta_qda_c = function(x0,class){
    subd = d[d$group==class,]
    muhat = mean(subd$x)
    pi_hat = mean(d$group==class)
    var_hat = var(subd$x)
    dlta = dnorm(x0,mean=muhat,sd=sqrt(var_hat))*pi_hat
    return(dlta)
}

In [ ]:
qda_pred = function(x0){
    deltas = sapply(1:3,function(c)delta_qda_c(x0,c))
    return(which.max(deltas))
}

In [ ]:
qda_pred(x0=1)

In [ ]:
qda_pred(x0=7)

In [ ]:
qda_pred(x0=20)

In [ ]:
x_seq = seq(-10,25,length.out=500)

In [ ]:
df = data.frame(x=x_seq,y_pred = sapply(x_seq,qda_pred))
df$y_pred = factor(df$y_pred)
df$c1 = sapply(x_seq,delta_qda_c,class=1)
df$c2 = sapply(x_seq,delta_qda_c,class=2)
df$c3 = sapply(x_seq,delta_qda_c,class=3)
head(df)

In [ ]:
mdf = melt(df,id.vars=c('x','y_pred'))
head(mdf)

In [ ]:
levels(mdf$variable) = c(1,2,3)

In [ ]:
ggplot(data=d,mapping=aes(x=x,color=group,group=group))+
    geom_density()+
    geom_point(mapping=aes(x=x,y=0))

In [ ]:
ggplot(data=mdf,mapping=aes(x=x,y=0,color=y_pred,group=y_pred))+
    geom_point()+
    geom_line(mapping=aes(x=x,y=value,group=variable,color=variable),inherit.aes=FALSE)

In [ ]:
library('MASS')
?qda

In [ ]:
my_preds = sapply(d$x,qda_pred)
my_preds

In [ ]:
mod = qda(group~.,data=d)

In [ ]:
mod

In [ ]:
muhat

In [ ]:
pi_hat

In [ ]:
mod_preds = predict(mod)$class
mod_preds

In [ ]:
all(mod_preds == my_preds)

In [ ]:
head(predict(mod)$posterior)

In [ ]:
mod_df = cbind(x_seq,predict(mod,newdata=data.frame(x=x_seq))$posterior)
colnames(mod_df)[1] = c('x')
mod_df = data.frame(mod_df)
mmod_df = melt(mod_df,id.vars='x')
levels(mmod_df$variable) = 1:3
head(mmod_df)

In [ ]:
ggplot(data=mdf,mapping=aes(x=x,y=0,color=y_pred,group=y_pred))+
    geom_point()+
    #geom_line(mapping=aes(x=x,y=value,group=variable,color=variable),inherit.aes=FALSE)+
    geom_line(mapping=aes(x=x,y=value,group=variable,color=variable),inherit.aes=FALSE,data=mmod_df)

In [ ]:
head(df)

In [ ]:
df[,3:5] = df[,3:5]/rowSums(df[,3:5])

In [ ]:
mdf = melt(df,id.vars=c('x','y_pred'))

In [ ]:
levels(mdf$variable) = c(1,2,3)

In [ ]:
ggplot(data=mdf,mapping=aes(x=x,y=0,color=y_pred,group=y_pred))+
    geom_point()+
    geom_line(mapping=aes(x=x,y=value,group=variable,color=variable),inherit.aes=FALSE)
    geom_line(mapping=aes(x=x,y=value,group=variable,color=variable),inherit.aes=FALSE,data=mmod_df)

In [ ]:
delta_qda_c2 = function(x0,class){
    subd = d[d$group==class,]
    muhat = mean(subd$x)
    pi_hat = mean(d$group==class)
    var_hat = var(subd$x)
    dlta = -1/2*log(var_hat) - 
        (x0-muhat)^2/(2*var_hat) +log(pi_hat)
    return(dlta)
}
qda_pred2 = function(x0){
    deltas = sapply(1:3,function(c)delta_qda_c2(x0,c))
    return(which.max(deltas))
}

In [ ]:
df = data.frame(x=x_seq,y_pred = sapply(x_seq,qda_pred2))
df$y_pred = factor(df$y_pred)
df$c1 = sapply(x_seq,delta_qda_c2,class=1)
df$c2 = sapply(x_seq,delta_qda_c2,class=2)
df$c3 = sapply(x_seq,delta_qda_c2,class=3)

mdf = melt(df,id.vars=c('x','y_pred'))
levels(mdf$variable) = c(1,2,3)

ggplot(data=mdf,mapping=aes(x=x,y=0,color=y_pred,group=y_pred))+
    geom_point(shape=1,size=1/10)+
    geom_line(mapping=aes(x=x,y=value,group=variable,color=variable),lwd=2)

# for $p>1$

In [ ]:
library('palmerpenguins')
penguins = penguins[complete.cases(penguins),]
head(penguins)

In [ ]:
d = penguins[,c('bill_length_mm','bill_depth_mm','species')]
head(d)

In [ ]:
plot_fit = function(v1,v2,df=penguins,N=floor(sqrt(10000)),scaleit=FALSE,fmla='species~.'){
    train_df = df[,c('species',v1,v2)]
    if(scaleit)
        train_df[,c(v1,v2)] = scale(train_df[,c(v1,v2)])
    
    #mod = knn3(species~.,data=train_df,k=k)
    #mod = lda(formula=as.formula(fmla),data=train_df)
    mod = qda(formula=as.formula(fmla),data=train_df)
    
    r1 = range(train_df[[v1]])
    r2 = range(train_df[[v2]])
    
    s1 = seq(r1[1],r1[2],length.out=N)
    s2 = seq(r2[1],r2[2],length.out=N)
    
    p_df = expand.grid(v1=s1,v2=s2)
    colnames(p_df) = c(v1,v2)
    preds = predict(mod,newdata=p_df)$class
    p_df$species = preds
    
    ggplot(data=p_df,mapping=aes_string(x=v1,y=v2,fill='species',shape='species'))+geom_tile()+
        geom_point(data=train_df,size=5)
}

In [ ]:
plot_fit(v1='bill_length_mm',v2='bill_depth_mm',df=penguins)

In [ ]:
plot_fit(v1='flipper_length_mm',v2='bill_depth_mm',df=penguins)

In [ ]:
plot_fit(v1='bill_length_mm',v2='bill_depth_mm',df=penguins,
             fmla='species~I(bill_length_mm^2)+I(bill_depth_mm^3)+I(bill_depth_mm^2)')

intercept?

In [ ]:
model.matrix(mod)

[source code for lda](https://github.com/cran/MASS/blob/master/R/lda.R) see line 29